# Importere data fra SSB med JSON-spørring

[Veiledning hos SSB](https://github.com/janbrus/ssb-api-python-examples/blob/master/jsonstatToPandas_funksjon.ipynb)

Koden her er hentet fra denne side for Tabell 10638


In [1]:
import json
import pandas as pd
import requests
# pyjstat er biblioteket for å lese JSON-stat, 'pip install pyjstat' 
from pyjstat import pyjstat

## Alternativ 1 - Lim inn POST Url og JSON-spørring

In [2]:
# Lim in PostUrl som spørringen skal postes mot
postUrl = "https://data.ssb.no/api/v0/no/table/10638/"


In [3]:
# Spørringen og endepunktet til API-et hentes fra Statistikkbanken eller API-konsoll. 
# apiQuery = # lim inn spørringen din her


apiQuery = {
  "query": [
    {
      "code": "KonsumInnd",
      "selection": {
        "filter": "item",
        "values": [
          "TOTK"
        ]
      }
    },
    {
      "code": "Reiseliv",
      "selection": {
        "filter": "item",
        "values": [
          "0",
          "01",
          "01.01",
          "01.02",
          "01.03",
          "01.04",
          "01.05",
          "01.06",
          "01.07",
          "01.08",
          "01.09",
          "01.10",
          "02"
        ]
      }
    },
    {
      "code": "ContentsCode",
      "selection": {
        "filter": "item",
        "values": [
          "FastePriser"
        ]
      }
    }
  ],
  "response": {
    "format": "json-stat2"
  }
}

In [4]:
# Funksjon for å konvertere uttrekket til 2 dataframes, tar 2 paramtre

def apiToDataframe(postUrl, query):

    # postUrl som spørringen skal postes mot
    # Spørringen og endepunktet til API-et kan hentes fra Statistikkbanken.

    res = requests.post(postUrl, json=query)
    # legger resultat i ds. DS har i tillegg en del metadata
    ds = pyjstat.Dataset.read(res.text)
    # skriver resultatet til to dataframes
    # først dataframe med tekst
    df = ds.write('dataframe')
    # deretter dataframe med koder
    df_id = ds.write('dataframe', naming='id')
    # returnerer også ds i tilfelle en trenger metadata
    return df, df_id, ds

In [5]:
df, df_id, ds = apiToDataframe(postUrl, apiQuery)

In [6]:
df

konsumentgruppe                         reiselivsnæring  \
0     Totalt konsum                      Turistkonsum i alt   
1     Totalt konsum                      Turistkonsum i alt   
2     Totalt konsum                      Turistkonsum i alt   
3     Totalt konsum                      Turistkonsum i alt   
4     Totalt konsum                      Turistkonsum i alt   
..              ...                                     ...   
203   Totalt konsum  Turistkonsum utenom reiselivsprodukter   
204   Totalt konsum  Turistkonsum utenom reiselivsprodukter   
205   Totalt konsum  Turistkonsum utenom reiselivsprodukter   
206   Totalt konsum  Turistkonsum utenom reiselivsprodukter   
207   Totalt konsum  Turistkonsum utenom reiselivsprodukter   

               statistikkvariabel    år     value  
0    Faste 2021-priser (mill. kr)  2007  159792.0  
1    Faste 2021-priser (mill. kr)  2008  160457.0  
2    Faste 2021-priser (mill. kr)  2009  153994.0  
3    Faste 2021-priser (mill. kr)  2010  158334.0  
4    Faste 2021-priser (mill. kr)  2011  163273.0  
..                            ...   ...       ...  
203  Faste 2021-priser (mill. kr)  2018   70140.0  
204  Faste 2021-priser (mill. kr)  2019   70971.0  
205  Faste 2021-priser (mill. kr)  2020   67170.0  
206  Faste 2021-priser (mill. kr)  2021   77561.0  
207  Faste 2021-priser (mill. kr)  2022       NaN  

[208 rows x 5 columns]

In [21]:
df.columns

Index(['konsumentgruppe', 'reiselivsnæring', 'statistikkvariabel', 'år',
       'value'],
      dtype='object')

In [23]:
df['reiselivsnæring'].unique()

array(['Turistkonsum i alt', 'Reiselivsprodukter i alt',
       'Overnattingstjenester', 'Serveringstjenester',
       'Transport med jernbane', 'Transport med buss og drosje',
       'Transport med skip og ferger', 'Transport med fly',
       'Utleie av transportmidler',
       'Reisebyrå- og reisearrangørtjenester',
       'Kultur- og underholdningstjenester',
       'Sports- og fritidstjenester',
       'Turistkonsum utenom reiselivsprodukter'], dtype=object)

## Alternativ 2 Lag en funksjon ("jsqToTab") for å hente data via PxWebApi fra en lagret API-spørring på fil

Etter at du har laget tabellen hos SSB trykk "Lagre API-spørring", og legg fila i samme katalog som du har Jupyter notatblokka.

In [7]:
import os

import urllib.request

urllib.request.urlretrieve('https://raw.githubusercontent.com/uit-sok-1006-v25/uit-sok-1006-v25.github.io/main/ssbapi_table_10638.json', 
                'ssbapi_table_10638.json')


savedApiQuery = json.load(open('ssbapi_table_10638.json'))


In [8]:
# Funksjonen tar 1-2 paramtre
# Leser filer på formen ssbapitable_03013.json, der 03013 er ID til tabellen. 


# LEGG MERKE TIL NESTE SETNING!
# Eksempelet forutsetter at filen med API-spørring ligger i samme katalog.
# DVS SAMME PLASS SOM DU LAGRER JUPYTER NOTATBLOKKEN DIN


# funksjon som tar id på lagret API-spørring som input
# lang kan angi språk, default er norsk
# funksjonen returnerer to dataframes, en med tekst og en med koder

def jsqToTab(qid, lang='no'):
    
    savedApiQuery = json.load(open('ssbapi_table_' + qid + '.json'))
    # Vi finner spørringen og Tabell ID
    apiQuery = savedApiQuery['queryObj']
    tabId = savedApiQuery['tableIdForQuery']
    # PostUrl som spørringen skal postes mot
    # Spørringen og endepunktet til API-et kan hentes fra Statistikkbanken.
    postUrl = "https://data.ssb.no/api/v0/" + lang + "/table/" + tabId
    res = requests.post(postUrl, json=apiQuery)
    # legger resultat i ds
    ds = pyjstat.Dataset.read(res.text)
    # skriver resultatet til to dataframes
    df_id = ds.write('dataframe', naming='id')
    df = ds.write('dataframe')

    return df, df_id

In [9]:
df, df_id = jsqToTab("10638") #sett inn 5-sifret tabellID fra filnavn f.eks. 03013 slik jsqToTab("03013")

In [10]:
df

konsumentgruppe                         reiselivsnæring  \
0     Totalt konsum                      Turistkonsum i alt   
1     Totalt konsum                      Turistkonsum i alt   
2     Totalt konsum                      Turistkonsum i alt   
3     Totalt konsum                      Turistkonsum i alt   
4     Totalt konsum                      Turistkonsum i alt   
..              ...                                     ...   
203   Totalt konsum  Turistkonsum utenom reiselivsprodukter   
204   Totalt konsum  Turistkonsum utenom reiselivsprodukter   
205   Totalt konsum  Turistkonsum utenom reiselivsprodukter   
206   Totalt konsum  Turistkonsum utenom reiselivsprodukter   
207   Totalt konsum  Turistkonsum utenom reiselivsprodukter   

               statistikkvariabel    år     value  
0    Faste 2021-priser (mill. kr)  2007  159792.0  
1    Faste 2021-priser (mill. kr)  2008  160457.0  
2    Faste 2021-priser (mill. kr)  2009  153994.0  
3    Faste 2021-priser (mill. kr)  2010  158334.0  
4    Faste 2021-priser (mill. kr)  2011  163273.0  
..                            ...   ...       ...  
203  Faste 2021-priser (mill. kr)  2018   70140.0  
204  Faste 2021-priser (mill. kr)  2019   70971.0  
205  Faste 2021-priser (mill. kr)  2020   67170.0  
206  Faste 2021-priser (mill. kr)  2021   77561.0  
207  Faste 2021-priser (mill. kr)  2022       NaN  

[208 rows x 5 columns]

In [11]:
df_id

KonsumInnd Reiseliv ContentsCode   Tid     value
0         TOTK        0  FastePriser  2007  159792.0
1         TOTK        0  FastePriser  2008  160457.0
2         TOTK        0  FastePriser  2009  153994.0
3         TOTK        0  FastePriser  2010  158334.0
4         TOTK        0  FastePriser  2011  163273.0
..         ...      ...          ...   ...       ...
203       TOTK       02  FastePriser  2018   70140.0
204       TOTK       02  FastePriser  2019   70971.0
205       TOTK       02  FastePriser  2020   67170.0
206       TOTK       02  FastePriser  2021   77561.0
207       TOTK       02  FastePriser  2022       NaN

[208 rows x 5 columns]

In [12]:
df1, df_id1 = jsqToTab("10638", 'en') #sett inn 5 sifret tabellID 

In [13]:
df1

consumer group                             tourism industry  \
0    Total tourism consumption   Total consumption expenditures by tourists   
1    Total tourism consumption   Total consumption expenditures by tourists   
2    Total tourism consumption   Total consumption expenditures by tourists   
3    Total tourism consumption   Total consumption expenditures by tourists   
4    Total tourism consumption   Total consumption expenditures by tourists   
..                         ...                                          ...   
203  Total tourism consumption  Total tourism consumption of other products   
204  Total tourism consumption  Total tourism consumption of other products   
205  Total tourism consumption  Total tourism consumption of other products   
206  Total tourism consumption  Total tourism consumption of other products   
207  Total tourism consumption  Total tourism consumption of other products   

                               contents  year     value  
0    Constant 2021 prices (NOK million)  2007  159792.0  
1    Constant 2021 prices (NOK million)  2008  160457.0  
2    Constant 2021 prices (NOK million)  2009  153994.0  
3    Constant 2021 prices (NOK million)  2010  158334.0  
4    Constant 2021 prices (NOK million)  2011  163273.0  
..                                  ...   ...       ...  
203  Constant 2021 prices (NOK million)  2018   70140.0  
204  Constant 2021 prices (NOK million)  2019   70971.0  
205  Constant 2021 prices (NOK million)  2020   67170.0  
206  Constant 2021 prices (NOK million)  2021   77561.0  
207  Constant 2021 prices (NOK million)  2022       NaN  

[208 rows x 5 columns]

In [14]:
df_id1

KonsumInnd Reiseliv ContentsCode   Tid     value
0         TOTK        0  FastePriser  2007  159792.0
1         TOTK        0  FastePriser  2008  160457.0
2         TOTK        0  FastePriser  2009  153994.0
3         TOTK        0  FastePriser  2010  158334.0
4         TOTK        0  FastePriser  2011  163273.0
..         ...      ...          ...   ...       ...
203       TOTK       02  FastePriser  2018   70140.0
204       TOTK       02  FastePriser  2019   70971.0
205       TOTK       02  FastePriser  2020   67170.0
206       TOTK       02  FastePriser  2021   77561.0
207       TOTK       02  FastePriser  2022       NaN

[208 rows x 5 columns]

# Nedlasting
Dersom du har ei fil som er på GitHub eller på din maskin i en annen katalog kan følgende kode være nyttig. Fra GitHub bruker du "raw" versjonen av fila. Min fil er her: https://raw.githubusercontent.com/uit-sok-1006-v25/uit-sok-1006-v25.github.io/main/ssbapi_table_10638.json 

In [15]:
import os

import urllib.request


In [16]:
def jsqToTab_remote(qid, lang='no'):
    urllib.request.urlretrieve('https://raw.githubusercontent.com/uit-sok-1006-v25/uit-sok-1006-v25.github.io/main/ssbapi_table_10638.json', 
                'ssbapi_table_10638.json')
    savedApiQuery = json.load(open('ssbapi_table_10638.json'))
    # Vi finner spørringen og Tabell ID
    apiQuery = savedApiQuery['queryObj']
    tabId = savedApiQuery['tableIdForQuery']
    # PostUrl som spørringen skal postes mot
    # Spørringen og endepunktet til API-et kan hentes fra Statistikkbanken.
    postUrl = "https://data.ssb.no/api/v0/" + lang + "/table/" + tabId
    res = requests.post(postUrl, json=apiQuery)
    # legger resultat i ds
    ds = pyjstat.Dataset.read(res.text)
    # skriver resultatet til to dataframes
    df_id = ds.write('dataframe', naming='id')
    df = ds.write('dataframe')

    return df, df_id

In [17]:
df, df_id = jsqToTab_remote("10638") 

In [18]:
df

konsumentgruppe                         reiselivsnæring  \
0     Totalt konsum                      Turistkonsum i alt   
1     Totalt konsum                      Turistkonsum i alt   
2     Totalt konsum                      Turistkonsum i alt   
3     Totalt konsum                      Turistkonsum i alt   
4     Totalt konsum                      Turistkonsum i alt   
..              ...                                     ...   
203   Totalt konsum  Turistkonsum utenom reiselivsprodukter   
204   Totalt konsum  Turistkonsum utenom reiselivsprodukter   
205   Totalt konsum  Turistkonsum utenom reiselivsprodukter   
206   Totalt konsum  Turistkonsum utenom reiselivsprodukter   
207   Totalt konsum  Turistkonsum utenom reiselivsprodukter   

               statistikkvariabel    år     value  
0    Faste 2021-priser (mill. kr)  2007  159792.0  
1    Faste 2021-priser (mill. kr)  2008  160457.0  
2    Faste 2021-priser (mill. kr)  2009  153994.0  
3    Faste 2021-priser (mill. kr)  2010  158334.0  
4    Faste 2021-priser (mill. kr)  2011  163273.0  
..                            ...   ...       ...  
203  Faste 2021-priser (mill. kr)  2018   70140.0  
204  Faste 2021-priser (mill. kr)  2019   70971.0  
205  Faste 2021-priser (mill. kr)  2020   67170.0  
206  Faste 2021-priser (mill. kr)  2021   77561.0  
207  Faste 2021-priser (mill. kr)  2022       NaN  

[208 rows x 5 columns]